In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torchvision
import torch 
import time 
import os

In [2]:
%cd yolo_tracking/

/Users/shreyaskumar/Documents/Shreyas/Work/yolo_v_8_testing/yolo_tracking


#### Necessary Functions for Post Processing

In [3]:
def get_distinct_object(dataframe):
    object_set = set()
    for idx, row in dataframe.iterrows():
        object_set.add(row['object_id'])
    return object_set

def get_distinct_objects_by_class(dataframe):
    object_set = set()
    object_class_tracker = {}

    for idx, row in dataframe.iterrows():
        if row['object_id'] in object_set:
            continue
        objet_set.add(row['object_id'])
        object_class_tracker[row['object_class']] == 0 if row['object_class'] not in object_class_tracker else object_class_tracker[row['object_class']] + 1

    return object_class_tracker
        
def distinct_object_counter(dataframe):
    num_of_distinct_objects = get_distinct_object(dataframe)
    return len(num_of_distinct_objects)

def detection_counter(dataframe):
    object_set = set()
    for idx, row in dataframe.iterrows():
        object_set.add((row['frame_id'], row['object_id']))
    return len(object_set)

def file_processor(experiment_files):
    number_of_distinct_objects = {}
    number_of_detections = {}
    for key, value in experiment_files.items():
        dataframe = pd.read_csv(value)
        number_of_distinct_objects[key] = distinct_object_counter(dataframe)
        number_of_detections[key] = detection_counter(dataframe)
    return (number_of_detections, number_of_distinct_objects)


def data_plotter(time_complexities, number_of_detections, num_of_distinct_objects, Model_Type):
    fig, (axs1, axs2, axs3) = plt.subplots(1, 3, figsize=(30,5))

    fig.suptitle(f'{Model_Type} Statisitcs', fontsize=20)
    
    axs1.plot(time_complexities.keys(), time_complexities.values())
    axs1.set_title('Time Complexity')
    axs1.set_xlabel('Algoritm frame processing rate')
    axs1.set_ylabel('Processing Time in seconds')
    
    axs2.bar(number_of_detections.keys(), number_of_detections.values())
    axs2.set_title('Number of Detections made')
    axs2.set_xlabel('Frame Processing Rate')
    axs2.set_ylabel('Number of Objects Identified')
    
    axs3.bar(num_of_distinct_objects.keys(), num_of_distinct_objects.values())
    axs3.set_title('Number of distinct objects identified')
    axs3.set_xlabel('Frame Processing Rate')
    axs3.set_ylabel('Number of Objects Identified')

    
def dataframe_concatenator(experiment_files, model_type):
    result_dataframe = pd.read_csv(experiment_files['1fps'])
    result_dataframe['model_type'] = model_type
    result_dataframe['vid_stride'] = '1fps'
    for key, value in experiment_files.items():
        if key == '1fps':
            continue
        dataframe = pd.read_csv(value)
        dataframe['model_type'] = 'yolov8s'
        dataframe['vid_stride'] = key
        result_dataframe = pd.concat([result_dataframe, dataframe], axis=0)
        
    result_dataframe
    result_dataframe.to_csv(f'combined_result_{model_type}.csv')


## Experimentation with YOLOv8s

The following set of experiments are being used to analyse the performance and accuracy of the BoxMot tracking algorithm for different variations of the video stride. Video stride in the context of object tracking refers to how many frames of the input video is being processed in 1 second, it is important you do not confuse this with the input video frame rate, because that frame rate refers to the rate at which the camera is taking snapshots of the scene to form the output video.

Processing the output video at 1fps will give us the most accurate form of object detection, because you are tracking at such a slow rate that it gives the YOLO algorithm it give the algorithm more time to analyse a frame in one second. Object tracking algorithms can sometimes perform better on videos with lower frame rates, but the relationship between frame rate and tracking performance is not straightforward. 

In [4]:
time_complexities_model_s = {}

### Processing Video output at 1 fps

In [5]:
start_time = time.time()
!python3 examples/track.py --yolo-model yolov8s --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 1 # bboxes only
end_time = time.time()

2024-03-15 13:49:50.852 | ERROR    | boxmot.utils.checks:check_packages:26 - The 'ultralytics@ git+https://github.com/mikel-brostrom/ultralytics.git' distribution was not found and is required by the application
2024-03-15 13:49:50.853 | WARNING  | boxmot.utils.checks:check_packages:29 - 
Missing packages: "ultralytics @ git+https://github.com/mikel-brostrom/ultralytics.git" 
Atempting installation...
2024-03-15 13:49:56.122 | SUCCESS  | boxmot.utils.checks:check_packages:35 - All the missing packages were installed successfully
yolov8s
False
Default Video Frame Rate: 1

Traceback (most recent call last):
  File "/Users/shreyaskumar/Documents/Shreyas/Work/yolo_v_8_testing/yolo_tracking/examples/track.py", line 213, in <module>
    run(opt)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/Users/shreyaskumar/Documents/Shreyas/Work/yolo_v_8_testing/yo

In [6]:
time_complexities_model_s['1fps'] = end_time-start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_s["1fps"]}')

Excution Time for baseline BoxMot Model: 12.445793867111206


### Processing Video output at 5 fps

In [7]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8s --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 5 # bboxes only
end_time = time.time()

zsh:1: command not found: python


In [8]:
time_complexities_model_s['5fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {end_time-start_time}')

Excution Time for baseline BoxMot Model: 0.1217198371887207


### Processing Video output at 10 fps

In [9]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8s --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 10 # bboxes only
end_time = time.time()

zsh:1: command not found: python


In [10]:
time_complexities_model_s['10fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {end_time-start_time}')

Excution Time for baseline BoxMot Model: 0.12117791175842285


### Processing Video output at 15 fps

In [11]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8s --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 15 # bboxes only
end_time = time.time()

zsh:1: command not found: python


In [12]:
time_complexities_model_s['15fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {end_time-start_time}')

Excution Time for baseline BoxMot Model: 0.12293577194213867


In [13]:
print(f'Performance of yolov8s at their respective time complexites is as follows:')
print(time_complexities)

Performance of yolov8s at their respective time complexites is as follows:


NameError: name 'time_complexities' is not defined

## Experimentation with YOLOv8m

In [ ]:
time_complexities_model_m = {}

### Processing Video output at 1 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8m --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 1 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_m['1fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {end_time-start_time}')

### Processing Video output at 5 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8m --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 5 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_m['5fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_m["5fps"]}')

### Processing Video output at 10 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8m --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 10 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_m['10fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_m["10fps"]}')

### Processing Video output at 15 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8m --show --save-mot --source='../footage/sample.mp44' --save --vid_stride 15 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_m['15fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_m["15fps"]}')

## Experimentation with YOLOv8l

In [ ]:
time_complexities_model_l = {}

### Processing Video output at 1 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8l --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 1 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_l['1fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_l["1fps"]}')

### Processing Video output at 5 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8l --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 5 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_l['5fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_l["5fps"]}')

### Processing Video output at 10 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8l --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 10 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_l['10fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_l["10fps"]}')

### Processing Video output at 15 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8l --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 15 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_l['15fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_l["15fps"]}')

## Experimentation with YOLOv8X

In [ ]:
time_complexities_model_x = {}

### Processing Video output at 1 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8x --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 1 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_x['1fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_x["1fps"]}')

### Processing Video output at 5 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8x --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 5 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_x['5fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_x["5fps"]}')

### Processing Video output at 10 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8x --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 10 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_x['10fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_x["10fps"]}')

### Processing Video output at 15 fps

In [ ]:
start_time = time.time()
!python examples/track.py --yolo-model yolov8x --show --save-mot --source='../footage/sample.mp4' --save --vid_stride 15 # bboxes only
end_time = time.time()

In [ ]:
time_complexities_model_x['15fps'] = end_time - start_time
print(f'Excution Time for baseline BoxMot Model: {time_complexities_model_x["15fps"]}')

## Visualising and Analysing the Model Performance 

In [ ]:
experiment_files_small = {
    '1fps': './runs/track/yolov8s_experiment_vid_stride_1/mot/yolov8s_tampines_sample_2_vid_stride_args_1_post_processed_results.csv',
    '5fps': './runs/track/yolov8s_experiment_vid_stride_5/mot/yolov8s_tampines_sample_2_vid_stride_args_5_post_processed_results.csv',
    '10fps': './runs/track/yolov8s_experiment_vid_stride_10/mot/yolov8s_tampines_sample_2_vid_stride_args_10_post_processed_results.csv',
    '15fps': './runs/track/yolov8s_experiment_vid_stride_15/mot/yolov8s_tampines_sample_2_vid_stride_args_15_post_processed_results.csv',
}

experiment_files_medium = {
    '1fps': './runs/track/yolov8m_experiment_vid_stride_1/mot/yolov8m_tampines_sample_2_vid_stride_args_1_post_processed_results.csv',
    '5fps': './runs/track/yolov8m_experiment_vid_stride_5/mot/yolov8m_tampines_sample_2_vid_stride_args_5_post_processed_results.csv',
    '10fps': './runs/track/yolov8m_experiment_vid_stride_10/mot/yolov8m_tampines_sample_2_vid_stride_args_10_post_processed_results.csv',
    '15fps': './runs/track/yolov8m_experiment_vid_stride_15/mot/yolov8m_tampines_sample_2_vid_stride_args_15_post_processed_results.csv',
}

experiment_files_large = {
    '1fps': './runs/track/yolov8l_experiment_vid_stride_1/mot/yolov8l_tampines_sample_2_vid_stride_args_1_post_processed_results.csv',
    '5fps': './runs/track/yolov8l_experiment_vid_stride_5/mot/yolov8l_tampines_sample_2_vid_stride_args_5_post_processed_results.csv',
    '10fps': './runs/track/yolov8l_experiment_vid_stride_10/mot/yolov8l_tampines_sample_2_vid_stride_args_10_post_processed_results.csv',
    '15fps': './runs/track/yolov8l_experiment_vid_stride_15/mot/yolov8l_tampines_sample_2_vid_stride_args_15_post_processed_results.csv',
}

experiment_files_extra_large = {
    '1fps': './runs/track/yolov8x_experiment_vid_stride_1/mot/yolov8x_tampines_sample_2_vid_stride_args_1_post_processed_results.csv',
    '5fps': './runs/track/yolov8x_experiment_vid_stride_5/mot/yolov8x_tampines_sample_2_vid_stride_args_5_post_processed_results.csv',
    '10fps': './runs/track/yolov8x_experiment_vid_stride_10/mot/yolov8x_tampines_sample_2_vid_stride_args_10_post_processed_results.csv',
    '15fps': './runs/track/yolov8x_experiment_vid_stride_15/mot/yolov8x_tampines_sample_2_vid_stride_args_15_post_processed_results.csv',
}


In [ ]:
number_of_detections_model_s, num_of_distinct_objects_model_s = file_processor(experiment_files_small)
number_of_detections_model_m, num_of_distinct_objects_model_m = file_processor(experiment_files_medium)
number_of_detections_model_l, num_of_distinct_objects_model_l = file_processor(experiment_files_large)
number_of_detections_model_x, num_of_distinct_objects_model_x = file_processor(experiment_files_extra_large)

In [ ]:
data_plotter(time_complexities_model_s, number_of_detections_model_s, num_of_distinct_objects_model_s, 'YOLOv8s Model Statistics')
data_plotter(time_complexities_model_m, number_of_detections_model_m, num_of_distinct_objects_model_m, 'YOLOv8m Model Statistics')
data_plotter(time_complexities_model_l, number_of_detections_model_l, num_of_distinct_objects_model_l, 'YOLOv8l Model Statistics')
data_plotter(time_complexities_model_x, number_of_detections_model_x, num_of_distinct_objects_model_x, 'YOLOv8x Model Statistics')

In [ ]:
dataframe_concatenator(experiment_files_small, 'yolov8s')
dataframe_concatenator(experiment_files_medium, 'yolov8m')
dataframe_concatenator(experiment_files_large, 'yolov8l')
dataframe_concatenator(experiment_files_extra_large, 'yolov8x')

In [ ]:
time_complexities_model_m